In [33]:
import os
from dotenv import load_dotenv
from llama_index.core import Document
from llama_index.core import Settings
from llama_index.core import SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.vector_stores.pinecone import PineconeVectorStore
from pinecone import Pinecone
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core import Settings
import chromadb
import pandas as pd
import re
load_dotenv()
llm = Gemini(api_key=os.environ["GOOGLE_API_KEY"],model="models/gemini-pro")
embed_model = GeminiEmbedding(api_key=os.environ["GOOGLE_API_KEY"],model_name="models/embedding-001")
Settings.llm = llm
Settings.embed_model = embed_model


# Load data from PDF
from llama_index.core import SimpleDirectoryReader




In [36]:
# Step 2: Load CSV
csv_file = "processed_file.csv"  # Replace with your CSV file
df = pd.read_csv(csv_file)

# Step 3: Chunk the Data (Row-wise)
documents = [
    Document(text=row["content"], metadata={"ArticleID": row["Article_Number"]})
    for _, row in df.iterrows()
]

[Document(id_='177cc2a5-cb74-407b-b015-359306f77ad3', embedding=None, metadata={'ArticleID': 'Article 1'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text=' Name and territory of the Union (1) India, that is Bharat, shall be a Union of States (2) The States and the territories thereof shall be as specified in the First Schedule (3) The territory of India shall comprise The territories of the States; the Union territories specified in the First Schedule; and such other territories as may be acquired', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='9d9f66a9-d59e-43bf-9b5f-926bbcf906aa', embedding=None, metadata={'ArticleID': 'Article 2'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text=' Admission or establishment of new States: Parliament may by law admit into the Union, or 

In [31]:
em=embed_model.get_text_embedding("hello world")


In [4]:
len(em)

768

In [40]:
documents

[Document(id_='177cc2a5-cb74-407b-b015-359306f77ad3', embedding=None, metadata={'ArticleID': 'Article 1'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text=' Name and territory of the Union (1) India, that is Bharat, shall be a Union of States (2) The States and the territories thereof shall be as specified in the First Schedule (3) The territory of India shall comprise The territories of the States; the Union territories specified in the First Schedule; and such other territories as may be acquired', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='9d9f66a9-d59e-43bf-9b5f-926bbcf906aa', embedding=None, metadata={'ArticleID': 'Article 2'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text=' Admission or establishment of new States: Parliament may by law admit into the Union, or 

In [17]:
print(len(documents))

2


In [33]:
# Create a client and a new collection
client = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = client.get_or_create_collection("constitution")

# Create a vector store
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Create a storage context
storage_context = StorageContext.from_defaults(vector_store=vector_store)


In [41]:
pinecone_client = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
pinecone_index = pinecone_client.Index("aiagent")
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(
    vector_store=vector_store
)

In [42]:

# Create an index from the documents and save it to the disk.
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context,show_progress=True,
)


Upserted vectors: 100%|██████████| 481/481 [00:07<00:00, 61.94it/s]


In [5]:
import os
from dotenv import load_dotenv
from llama_index.llms.gemini import Gemini
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings
from llama_index.core import PromptTemplate
from llama_index.core.prompts.prompt_type import PromptType
import chromadb
from pinecone import Pinecone
from llama_index.vector_stores.pinecone import PineconeVectorStore


In [6]:
load_dotenv()
llm = Gemini(api_key=os.environ["GOOGLE_API_KEY"],model="models/gemini-1.5-pro-002")
embed_model = GeminiEmbedding(api_key=os.environ["GOOGLE_API_KEY"],model_name="models/embedding-001")

Settings.llm = llm
Settings.embed_model = embed_model

# # Load from disk
# load_client = chromadb.PersistentClient(path="./chroma_db")

# # Fetch the collection
# chroma_collection = load_client.get_collection("constitution")

# # Fetch the vector store
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

pinecone_client = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
pinecone_index = pinecone_client.Index("constitution")
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)


# Get the index from the vector store
index = VectorStoreIndex.from_vector_store(
    vector_store
)


In [7]:
pinecone_client.list_collections()

{'collections': []}

In [8]:
DEFAULT_TEXT_QA_PROMPT_TMPL = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query.\n"
    "Query: {query_str}\n"
    "Answer: "
    )

DEFAULT_TEXT_QA_PROMPT = PromptTemplate(
        DEFAULT_TEXT_QA_PROMPT_TMPL, 
        prompt_type=PromptType.QUESTION_ANSWER
        
    )

query_engine = index.as_query_engine(
    )

In [9]:
query = "can India be called bharat and by which article"
response = query_engine.query(query)
print(response)
print("\n")

Article 1 of the Constitution states that India, that is Bharat, shall be a Union of States.





In [10]:
res = llm.complete(query)
print(res)

India can be called Bharat.  This is established by **Article 1(1)** of the Constitution of India, which states:  "India, that is Bharat, shall be a Union of States."



In [27]:
import pandas as pd

df = pd.read_json("hf://datasets/nisaar/Constitution_of_India/constitution_train.jsonl", lines=True)

In [28]:
# import metrics
from ragas.metrics import (
    Faithfulness,
    AnswerRelevancy,
    ContextPrecision,
    ContextRecall,
)

# init metrics with evaluator LLM
from ragas.llms import LlamaIndexLLMWrapper

evaluator_llm = LlamaIndexLLMWrapper(Gemini(api_key=os.environ["GOOGLE_API_KEY"],model="models/gemini-1.5-pro-002"))
metrics = [
    Faithfulness(llm=evaluator_llm),
    AnswerRelevancy(llm=evaluator_llm),
    ContextPrecision(llm=evaluator_llm),
    ContextRecall(llm=evaluator_llm),
]

In [31]:
# convert to HF dataset

ds_dict = df.to_dict()
ds_dict['ground_truth'] = ds_dict.pop('answer')
ds_dict.keys()

dict_keys(['question', 'ground_truth'])

In [35]:
from ragas import EvaluationDataset

# Convert DataFrame to EvaluationDataset
evaluation_dataset = EvaluationDataset.from_pandas(df)


In [30]:
from ragas.integrations.llama_index import evaluate

result = evaluate(
    query_engine=query_engine,
    metrics=metrics,
    dataset=evaluation_dataset,
    llm=evaluator_llm,
    embeddings=GeminiEmbedding(api_key=os.environ["GOOGLE_API_KEY"],model_name="models/embedding-001")
)

Running Query Engine:   0%|          | 0/933 [00:00<?, ?it/s]Exception raised in Job[0]: ValidationError(2 validation errors for QueryStartEvent
query.str
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
query.QueryBundle
  Input should be a dictionary or an instance of QueryBundle [type=dataclass_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.9/v/dataclass_type)
Exception raised in Job[1]: ValidationError(2 validation errors for QueryStartEvent
query.str
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
query.QueryBundle
  Input should be a dictionary or an instance of QueryBundle [type=dataclass_type, input_value=None, input_type=NoneType]
    For further information visit https://err

AttributeError: 'float' object has no attribute 'response'

In [1]:
import pandas as pd

# Input and output file paths
input_file = "data\Constitution Of India.csv"  # Replace with the path to your input CSV file
output_file = "testdata\Constitution(updated).csv"  # Replace with the desired output file path

# Common word to add
common_word = "Article"

# Read the CSV file
df = pd.read_csv(input_file, header=None, names=["Text"])

# Add the common word to each row
df["Text"] = common_word + " " + df["Text"]

# Save the updated DataFrame to a new CSV
df.to_csv(output_file, index=False, header=False)

print(f"Updated CSV saved to {output_file}")


FileNotFoundError: [Errno 2] No such file or directory: 'data\\Constitution Of India.csv'

In [6]:
import pandas as pd
input_file = r"C:\Users\asifi\Desktop\prodigal\ragProgogal\data\Constitution(updated).csv"  
df = pd.read_csv(input_file, header=None, names=["Text"])

In [13]:
print(df["Text"][2])

Article 2. Admission or establishment of new States: Parliament may by law admit into the Union, or establish, new States on such terms and conditions, as it thinks fit


In [29]:
# Read the CSV file without header
input_file = "../data/Constitution(updated).csv"
df_no_header = pd.read_csv(input_file, header=None)

# Define the new header
new_header = ["Article"]

# Add the header to the DataFrame
df_no_header.columns = new_header

# Save the updated DataFrame to a new CSV file
output_file_with_header = "../data/Constitution(updated).csv"
df_no_header.to_csv(output_file_with_header, index=False)

print(f"Updated CSV with header saved to {output_file_with_header}")

Updated CSV with header saved to ../data/Constitution(updated).csv


In [17]:
df

,Articles,Article_Number
0,Article 1. Name and territory of the Union (1)...,Article 1
1,Article 2. Admission or establishment of new S...,Article 2
2,Article 2A. Sikkim to be associated with the U...,Article 2A
3,Article 3. Formation of new States and alterat...,Article 3
4,Article 4. Laws made under Articles 2 and 3 to...,Article 4
...,...,...
450,Article 378A. Special provision as to duration...,Article 378A
451,Article 392. Power of the President to remove ...,Article 392
452,Article 393. Short title This Constitution may...,Article 393
453,Article 394. Commencement This article and Art...,Article 394


In [27]:
import pandas as pd

# Load the CSV file
file_path = '../data/Constitution(updated).csv'  # Replace with the path to your CSV file
df = pd.read_csv(file_path)



# Extract the Article number and the remaining text
df['Article_Number'] = df['Articles'].str.extract(r'(Article \d+\w*)')
df['Articles'] = df['Articles'].str.replace(r'^Article \d+[a-zA-Z]?\.', '', regex=True)
columns = df.columns.tolist()
columns[0], columns[1] = columns[1], columns[0]  # Swap the first two columns
df = df[columns]
# Save the result to a new CSV file
output_path = '../data/processed_file.csv'  # Replace with desired output file name
df.to_csv(output_path, index=False)

print(f"Processed file saved to: {output_path}")


Processed file saved to: ../data/processed_file.csv


In [28]:
df
# df['Articles'] = df['Articles'].str.replace(r'^Article \d+[a-zA-Z]?\.', '', regex=True)
# df

,Article_Number,Articles
0,Article 1,"Name and territory of the Union (1) India, th..."
1,Article 2,Admission or establishment of new States: Par...
2,Article 2A,Sikkim to be associated with the Union Rep by...
3,Article 3,Formation of new States and alteration of are...
4,Article 4,Laws made under Articles 2 and 3 to provide f...
...,...,...
450,Article 378A,Special provision as to duration of Andhra Pr...
451,Article 392,Power of the President to remove difficulties...
452,Article 393,Short title This Constitution may be called t...
453,Article 394,"Commencement This article and Articles 5, 6, ..."


,Article_Number,Articles
0,Article 1,Article 1. Name and territory of the Union (1)...
1,Article 2,Article 2. Admission or establishment of new S...
2,Article 2A,Article 2A. Sikkim to be associated with the U...
3,Article 3,Article 3. Formation of new States and alterat...
4,Article 4,Article 4. Laws made under Articles 2 and 3 to...
...,...,...
450,Article 378A,Article 378A. Special provision as to duration...
451,Article 392,Article 392. Power of the President to remove ...
452,Article 393,Article 393. Short title This Constitution may...
453,Article 394,Article 394. Commencement This article and Art...


In [3]:
import pandas as pd
output_path = '../data/processed_file.csv'
file_path = '../data/Constitution(updated).csv'

df = pd.read_csv(file_path, names=['Content'])

# Extract the article number and the rest of the string into separate columns
df['Article Number'] = df['Content'].str.extract(r'Article (\d+)')
df['Content'] = df['Content'].str.replace(r'Article \d+', '', regex=True).str.strip()

# Save the result back to a new CSV
df.to_csv(output_path, index=False)

print(df)


                                               Content Article Number
0                                             Articles            NaN
1    . Name and territory of the Union (1) India, t...              1
2    . Admission or establishment of new States: Pa...              2
3    A. Sikkim to be associated with the Union Rep ...              2
4    . Formation of new States and alteration of ar...              3
..                                                 ...            ...
451  A. Special provision as to duration of Andhra ...            378
452  . Power of the President to remove difficultie...            392
453  . Short title This Constitution may be called ...            393
454  . Commencement This article and Articles 5, 6,...            394
455  . Repeals The Indian Independence Act, 1947 , ...            395

[456 rows x 2 columns]


In [ ]:
import pandas as pd
output_path = '../data/processed_file.csv'
file_path = '../data/Constitution(updated).csv'
# Load the CSV into a DataFrame
# Replace 'your_file.csv' with the path to your CSV file
df = pd.read_csv(file_path, names=['Content'])

# Extract the article number (number + optional letters) and the rest of the string into separate columns
df['Article Identifier'] = df['Content'].str.extract(r'Article (\d+\w*)')
df['Content'] = df['Content'].str.replace(r'Article \d+\w*\.', '', regex=True).str.strip()

# Save the result back to a new CSV
df.to_csv(file_path, index=False)

print(df)


                                               Content Article Identifier
0                                             Articles                NaN
1    Name and territory of the Union (1) India, tha...                  1
2    Admission or establishment of new States: Parl...                  2
3    Sikkim to be associated with the Union Rep by ...                 2A
4    Formation of new States and alteration of area...                  3
..                                                 ...                ...
451  Special provision as to duration of Andhra Pra...               378A
452  Power of the President to remove difficulties\...                392
453  Short title This Constitution may be called th...                393
454  Commencement This article and Articles 5, 6, 7...                394
455  Repeals The Indian Independence Act, 1947 , an...                395

[456 rows x 2 columns]


In [9]:
file_path = '../data/Constitution(updated).csv'
df = pd.read_csv(file_path)

In [10]:
df

,Articles
0,Article 1. Name and territory of the Union (1)...
1,Article 2. Admission or establishment of new S...
2,Article 2A. Sikkim to be associated with the U...
3,Article 3. Formation of new States and alterat...
4,Article 4. Laws made under Articles 2 and 3 to...
...,...
450,Article 378A. Special provision as to duration...
451,Article 392. Power of the President to remove ...
452,Article 393. Short title This Constitution may...
453,Article 394. Commencement This article and Art...


In [14]:
df['Articles'].str.extract(r'(Article \d+\w*)')

,0
0,Article 1
1,Article 2
2,Article 2A
3,Article 3
4,Article 4
...,...
450,Article 378A
451,Article 392
452,Article 393
453,Article 394
